In [1]:
import numpy as np
import matplotlib.pyplot as plt
from medpy import io
from radiomics import featureextractor, imageoperations
import six
import os
import pandas as pd
import SimpleITK as sitk
import seaborn as sns
from sklearn.metrics import roc_auc_score
import radiomics


import sys
sys.path.append('../../')


%load_ext autoreload
%autoreload 2
from radiomics_util import *

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)


/u/home/seel/.conda/envs/conda-adlm/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
gt = pd.read_csv("/data1/practical-sose23/morphometric/picai_labels/clinical_information/marksheet.csv")
gt

,patient_id,study_id,mri_date,patient_age,psa,psad,prostate_volume,histopath_type,lesion_GS,lesion_ISUP,case_ISUP,case_csPCa
0,10000,1000000,2019-07-02,73,7.70,NaN,55.0,MRBx,0+0,0,0,NO
1,10001,1000001,2016-05-27,64,8.70,0.09,102.0,NaN,NaN,NaN,0,NO
2,10002,1000002,2021-04-18,58,4.20,0.06,74.0,NaN,NaN,NaN,0,NO
3,10003,1000003,2019-04-05,72,13.00,NaN,71.5,SysBx,0+0,0,0,NO
4,10004,1000004,2020-10-21,67,8.00,0.10,78.0,SysBx+MRBx,"0+0,0+0","0,0",0,NO
...,...,...,...,...,...,...,...,...,...,...,...,...
1493,11470,1001494,2018-09-22,60,9.10,NaN,47.0,MRBx,"0+0,0+0","0,0",0,NO
1494,11471,1001495,2012-08-25,71,12.50,0.21,62.0,MRBx,"3+4,N/A,3+3","2,1",2,YES
1495,11472,1001496,2019-06-28,81,5.28,0.12,44.0,SysBx+MRBx,3+4,2,2,YES
1496,11473,1001497,2017-09-24,56,29.60,0.34,87.0,MRBx,0+0,0,0,NO


In [3]:
df1 = pd.read_pickle("../radiomics/df_t2w_all_all-features_resample_bias-corrected.pkl")
df2 = pd.read_pickle("../radiomics/df_adc_all_all-features_resample_bias-corrected.pkl")
df1

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_gldm_LargeDependenceHighGrayLevelEmphasis,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,ellipse_vol,patient_id,study_id,prostate_volume
1,v3.0.1,1.20.3,2.2.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},b7e07a24b98b72b6783555843e1c8df3ee92c986,3D,"(0.28125, 0.28125, 3.2999999825580133)",...,13943.148017817188,0.6331386704433819,0.007370307806137414,0.04544061435931626,11.636874745836023,0.00029631336438916357,31.627291,10000,1000000,55.0
1,v3.0.1,1.20.3,2.2.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},506f40b3dd1be17b2bfe821bcfcf7864fcc412b1,3D,"(0.30000001192093, 0.30000001192093, 3.6000000...",...,11233.75609535354,0.5935250469041303,0.007952367162358927,0.041430117006279965,7.4169872044022505,0.0003385122084686867,88.094831,10001,1000001,102.0
1,v3.0.1,1.20.3,2.2.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},f7152603cd28a4571523d8ad09de1693032aca0c,3D,"(0.5, 0.5, 3.000000075935057)",...,8345.461382310685,0.22505014119304018,0.006597650757225366,0.1006349365690926,24.71857791368083,0.000706770048292158,59.557682,10002,1000002,74.0
1,v3.0.1,1.20.3,2.2.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},4a78c3a51494a7e2318643b480cfcbc52fd99a45,3D,"(0.5, 0.5, 3.000000080342142)",...,6514.693153017427,0.6678222348024903,0.011376834340760863,0.059340941033063956,8.616535047923623,0.0006878207755164997,61.460102,10003,1000003,71.5
1,v3.0.1,1.20.3,2.2.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},6acbf87411c739e33cdc4b3be587ebb88738a2a2,3D,"(0.30000001192093, 0.30000001192093, 3.5999999...",...,7067.529045081282,3.6482982781461666,0.025084888142404373,0.02209866795539414,1.5064993578636905,0.000562019400089227,59.366459,10004,1000004,78.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,v3.0.1,1.20.3,2.2.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},263acfd0a27332fc99becebd8301b09163000c07,3D,"(0.5, 0.5, 3.5000000091804995)",...,8764.054696829162,0.744365014585416,0.010738500830972837,0.048682693403121265,7.559701094010482,0.0005153198387689947,45.555510,11471,1001495,62.0
1,v3.0.1,1.20.3,2.2.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},686aac1fd169cd485ed3edda4cb8ce5876915417,3D,"(0.30000001192093, 0.30000001192093, 3.5999999...",...,5537.745182671091,20.28184681445372,0.0674635792119614,0.008615858585351532,0.20749396659857963,0.0006496413680494103,29.612047,11472,1001496,44.0
1,v3.0.1,1.20.3,2.2.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},3402dc991b36399113689938ff0b85273a9b0264,3D,"(0.30000001192093, 0.30000001192093, 3.6000000...",...,10745.400786459772,0.7994737965687021,0.009042691426220096,0.045653201258894084,8.666895433043111,0.00037103949464335946,58.687654,11473,1001497,87.0
1,v3.0.1,1.20.3,2.2.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},f12b387cd19f1f27544e353d060897f12e8e36fe,3D,"(0.625, 0.625, 3.599999876696945)",...,8060.871251431122,0.5586066861557434,0.009425688135718134,0.050971627092910046,7.894140990518342,0.0005101327840272178,75.624410,11474,1001498,83.0


In [4]:

merged_df = df1.merge(df2, on='study_id', suffixes=("_t2w","_adc"))
merged_df = merged_df.merge(gt[["study_id", "case_csPCa"]], on="study_id")
merged_df['case_csPCa'] = merged_df['case_csPCa'].map({'YES': 1, 'NO': 0})

columns_to_drop = merged_df.filter(regex='^diagnostics_').columns
merged_df = merged_df.drop(columns=columns_to_drop)
#merged_df = merged_df.drop(columns=["lesion_GS", "lesion_ISUP", "case_ISUP", "mri_date","histopath_type", "psa", "psad", "study_id", "prostate_volume_x", "prostate_volume_y","patient_age"])
#merged_df = merged_df.dropna()

data = merged_df.iloc[:,0:-1]
target = merged_df["case_csPCa"]
target


0       0
1       0
2       0
3       0
4       0
       ..
1493    0
1494    1
1495    1
1496    0
1497    0
Name: case_csPCa, Length: 1498, dtype: int64

In [5]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Standardize the dataframe
data_standardized = scaler.fit_transform(data)
# Convert the standardized array back to a dataframe
data_standardized = pd.DataFrame(data_standardized, columns=data.columns)

In [6]:
df = pd.concat((data_standardized, target), axis = 1)
df = df.dropna()
df

,original_shape_Elongation_t2w,original_shape_Flatness_t2w,original_shape_LeastAxisLength_t2w,original_shape_MajorAxisLength_t2w,original_shape_Maximum2DDiameterColumn_t2w,original_shape_Maximum2DDiameterRow_t2w,original_shape_Maximum2DDiameterSlice_t2w,original_shape_Maximum3DDiameter_t2w,original_shape_MeshVolume_t2w,original_shape_MinorAxisLength_t2w,...,original_gldm_LargeDependenceHighGrayLevelEmphasis_adc,original_gldm_LargeDependenceLowGrayLevelEmphasis_adc,original_gldm_LowGrayLevelEmphasis_adc,original_gldm_SmallDependenceEmphasis_adc,original_gldm_SmallDependenceHighGrayLevelEmphasis_adc,original_gldm_SmallDependenceLowGrayLevelEmphasis_adc,ellipse_vol_adc,patient_id_adc,prostate_volume_adc,case_csPCa
0,-1.176000,-1.101958,-0.901461,-0.403932,-0.389867,-0.881783,-0.405019,-0.618087,-0.710789,-0.931939,...,0.431070,-0.544478,-0.506786,1.417578,0.643784,-0.449350,-0.699647,-1.729393,-0.276964,0
1,-0.015221,0.550957,1.299101,1.032387,1.213382,0.939687,0.929445,0.933943,1.052948,0.979412,...,0.648873,-0.584954,-0.603059,-1.189990,0.489077,-0.665806,1.432540,-1.727039,1.012842,0
2,1.375186,1.160742,0.645038,0.020753,0.090274,0.320773,0.108989,0.094198,0.323349,0.744141,...,0.300535,-0.333334,-0.290130,0.601531,0.264757,-0.182141,0.244424,-1.724686,0.244447,0
3,1.339354,1.130609,0.682782,0.073439,-0.169272,0.514633,0.146228,0.174977,0.381403,0.781736,...,0.634743,-0.587858,-0.601643,-0.720251,0.522346,-0.632283,0.257145,-1.722333,0.175841,0
4,-0.504956,-1.141670,-0.162673,0.531956,0.669646,0.164962,0.707184,0.279253,0.033214,0.231382,...,0.616935,-0.592940,-0.602930,-0.532379,0.560596,-0.644147,0.494082,-1.719980,0.354218,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493,-0.050177,0.366846,-0.519830,-0.762373,-0.537237,-0.695910,-0.618994,-0.695255,-0.660150,-0.729638,...,-1.702481,1.690576,1.701523,-0.037102,-1.693614,1.710531,-0.816963,1.729867,-0.496505,0
1494,-0.063947,0.738919,0.245097,-0.158225,0.252403,-0.080097,0.341220,-0.038891,-0.230682,-0.168400,...,-1.698242,1.707553,1.701523,-0.312658,-1.701399,1.678981,-0.091293,1.732220,-0.084865,1
1495,-1.681959,-1.529316,-0.997232,-0.256370,-0.364573,-1.098420,-0.311429,-0.574478,-0.732111,-1.057232,...,0.575280,-0.603340,-0.602762,0.293850,0.651579,-0.618075,-0.502283,1.734574,-0.578833,1
1496,0.475333,0.183247,0.433988,0.350741,-0.002606,0.783978,0.062435,0.516636,0.341298,0.604571,...,-1.690848,1.737162,1.701523,-0.853482,-1.716679,1.617059,0.406475,1.736927,0.601202,0


In [7]:
df.iloc[:,:-1]

,original_shape_Elongation_t2w,original_shape_Flatness_t2w,original_shape_LeastAxisLength_t2w,original_shape_MajorAxisLength_t2w,original_shape_Maximum2DDiameterColumn_t2w,original_shape_Maximum2DDiameterRow_t2w,original_shape_Maximum2DDiameterSlice_t2w,original_shape_Maximum3DDiameter_t2w,original_shape_MeshVolume_t2w,original_shape_MinorAxisLength_t2w,...,original_gldm_LargeDependenceEmphasis_adc,original_gldm_LargeDependenceHighGrayLevelEmphasis_adc,original_gldm_LargeDependenceLowGrayLevelEmphasis_adc,original_gldm_LowGrayLevelEmphasis_adc,original_gldm_SmallDependenceEmphasis_adc,original_gldm_SmallDependenceHighGrayLevelEmphasis_adc,original_gldm_SmallDependenceLowGrayLevelEmphasis_adc,ellipse_vol_adc,patient_id_adc,prostate_volume_adc
0,-1.176000,-1.101958,-0.901461,-0.403932,-0.389867,-0.881783,-0.405019,-0.618087,-0.710789,-0.931939,...,-2.051866,0.431070,-0.544478,-0.506786,1.417578,0.643784,-0.449350,-0.699647,-1.729393,-0.276964
1,-0.015221,0.550957,1.299101,1.032387,1.213382,0.939687,0.929445,0.933943,1.052948,0.979412,...,1.174553,0.648873,-0.584954,-0.603059,-1.189990,0.489077,-0.665806,1.432540,-1.727039,1.012842
2,1.375186,1.160742,0.645038,0.020753,0.090274,0.320773,0.108989,0.094198,0.323349,0.744141,...,-0.589869,0.300535,-0.333334,-0.290130,0.601531,0.264757,-0.182141,0.244424,-1.724686,0.244447
3,1.339354,1.130609,0.682782,0.073439,-0.169272,0.514633,0.146228,0.174977,0.381403,0.781736,...,0.864760,0.634743,-0.587858,-0.601643,-0.720251,0.522346,-0.632283,0.257145,-1.722333,0.175841
4,-0.504956,-1.141670,-0.162673,0.531956,0.669646,0.164962,0.707184,0.279253,0.033214,0.231382,...,0.448493,0.616935,-0.592940,-0.602930,-0.532379,0.560596,-0.644147,0.494082,-1.719980,0.354218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493,-0.050177,0.366846,-0.519830,-0.762373,-0.537237,-0.695910,-0.618994,-0.695255,-0.660150,-0.729638,...,-0.250832,-1.702481,1.690576,1.701523,-0.037102,-1.693614,1.710531,-0.816963,1.729867,-0.496505
1494,-0.063947,0.738919,0.245097,-0.158225,0.252403,-0.080097,0.341220,-0.038891,-0.230682,-0.168400,...,0.134743,-1.698242,1.707553,1.701523,-0.312658,-1.701399,1.678981,-0.091293,1.732220,-0.084865
1495,-1.681959,-1.529316,-0.997232,-0.256370,-0.364573,-1.098420,-0.311429,-0.574478,-0.732111,-1.057232,...,-0.498182,0.575280,-0.603340,-0.602762,0.293850,0.651579,-0.618075,-0.502283,1.734574,-0.578833
1496,0.475333,0.183247,0.433988,0.350741,-0.002606,0.783978,0.062435,0.516636,0.341298,0.604571,...,0.807230,-1.690848,1.737162,1.701523,-0.853482,-1.716679,1.617059,0.406475,1.736927,0.601202


In [8]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import cross_validate

In [12]:

# Initialize the Lasso model
lasso = LogisticRegression(penalty="l2", max_iter=1000)  # You can adjust the value of alpha (regularization strength) as needed

result = cross_validate(lasso, X=df.iloc[:,:-1], y=df["case_csPCa"], cv=5, scoring="roc_auc", return_train_score=True, return_estimator=True)
print(result)
estimators = result["estimator"]
test_scores = result["test_score"]
train_scores = result["train_score"]
print("test scores: ", test_scores.mean())
print("train scores:", train_scores.mean())

#features = np.asarray([])
#for estimator in estimators:
#    feature_importances = estimator.coef_
#    # Create a mask to identify the selected features
#    selected_features = data_standardized.columns[feature_importances != 0]
#    features = np.concatenate((features, selected_features))
#    # Print the selected features
#    print("Selected features:", list(selected_features))

{'fit_time': array([0.09232426, 0.08730459, 0.08500719, 0.08945322, 0.09283495]), 'score_time': array([0.00362086, 0.00319672, 0.00319386, 0.00324774, 0.00319862]), 'estimator': [LogisticRegression(max_iter=1000), LogisticRegression(max_iter=1000), LogisticRegression(max_iter=1000), LogisticRegression(max_iter=1000), LogisticRegression(max_iter=1000)], 'test_score': array([0.76744715, 0.69299356, 0.64990796, 0.67909553, 0.70541883]), 'train_score': array([0.78265277, 0.80178983, 0.80899631, 0.80265822, 0.79620407])}
test scores:  0.6989726052954575
train scores: 0.7984602409679851


In [34]:
names, freq = np.unique(features, return_counts=True)
result = pd.DataFrame([names, freq]).T
result.sort_values(1)

,0,1
15,original_shape_MinorAxisLength_t2w,1
14,original_shape_MinorAxisLength_adc,1
5,original_gldm_LargeDependenceLowGrayLevelEmpha...,1
7,original_glrlm_LongRunLowGrayLevelEmphasis_adc,1
11,original_glszm_SizeZoneNonUniformity_adc,1
6,original_glrlm_LongRunEmphasis_adc,2
16,original_shape_Sphericity_t2w,2
9,original_glrlm_ShortRunHighGrayLevelEmphasis_adc,2
10,original_glrlm_ShortRunLowGrayLevelEmphasis_adc,2
0,ellipse_vol_adc,3
